# License Plate Detection - Data Collection and Exploration
Welcome to the license plate detection! This notebook explains step-by-step how to train and use a license plate detection model for the Machine Learning Engineer Capstone Project. Make sure you meet all preconditions before you start, see README.md.

### Imports

In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import matplotlib

from distutils.version import StrictVersion
from matplotlib import pyplot as plt
from PIL import Image

Test Tensorflow version...

In [3]:
print ("Tensorflow: {}".format(tf.__version__))

if StrictVersion(tf.__version__) < StrictVersion('1.13.1'):
    raise ImportError('Please upgrade your TensorFlow installation to v1.13.1')

if StrictVersion(tf.__version__) >= StrictVersion('2.0.0'):
    raise ImportError('Please downgrade your TensorFlow installation to v1.13.*.')

device_name = tf.test.gpu_device_name()

if device_name != "/device:GPU:0":
    raise SystemError("GPU device not found")

print(f"GPU device: {device_name}")

Tensorflow: 1.13.1
Default GPU Device: /device:GPU:0


### Env setup

In [4]:
# This is needed to display the images inside the notebook cells.
%matplotlib inline

os.chdir(os.path.join(os.getcwd(), 'tf_object_detection'))
working_dir = os.getcwd()

sys.path.append(working_dir)
sys.path.append(working_dir + "/slim")

path = working_dir + ';' + working_dir + '/slim' + ';' + working_dir + '/object_detection'
os.environ['PYTHONPATH'] = path

os.chdir(os.path.join(os.getcwd(), 'object_detection'))

### Object detection imports
Here are the imports from the object detection module.

In [5]:
import object_detection

### Model preparation

#### Variables
Define the pretrained model to be used.
Change MODEL and PIPELINE_CONFING_FILENAME if you want to use other pretrained models, see the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [6]:
# choose pretrained model
MODEL = 'ssdlite_mobilenet_v2_coco_2018_05_09'
PIPELINE_CONFING_FILENAME = 'ssdlite_mobilenet_v2_coco.config'

#MODEL = 'ssd_mobilenet_v2_coco_2018_03_29'
#PIPELINE_CONFING_FILENAME = 'ssd_mobilenet_v2.config'

NUM_CLASSES = 1

In [7]:
# ****************** don't edit ******************

# What model to download.
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
PRETRAIND_MODELS_DIR = 'models'

# List of the strings that is used to add correct label for each box.
DETECTION_COFIG_DIR = '../../config/plate_detection'
PATH_TO_LABELS = os.path.join(DETECTION_COFIG_DIR, 'labels_map.pbtxt')

PIPELINE_CONFING_FILEPATH = os.path.join(DETECTION_COFIG_DIR, PIPELINE_CONFING_FILENAME)

OUTPUT_DIR = '../../output/plate_detection'
CHECKPOINTS_DIR = OUTPUT_DIR + '/checkpoints/' + MODEL
FINETUNED_MODEL_DIR = OUTPUT_DIR + '/fine_tuned_models/' + MODEL

# Path to frozen detection graph.
PATH_TO_FROZEN_GRAPH = os.path.join(FINETUNED_MODEL_DIR, 'frozen_inference_graph.pb')

TFLITE_MODEL_PATH = os.path.join(OUTPUT_DIR, 'glpd-model-float.tflite')

print ('Labels file:           {}'.format(PATH_TO_LABELS))
print ('Pipeline config file:  {}'.format(PIPELINE_CONFING_FILEPATH))
print ('Checkpoints directory: {}'.format(CHECKPOINTS_DIR))
print ('Frozen model file:     {}'.format(PATH_TO_FROZEN_GRAPH))
print ('TFLite model file:     {}'.format(TFLITE_MODEL_PATH))

Labels file:           ../../config/plate_detection\labels_map.pbtxt
Pipeline config file:  ../../config/plate_detection\ssdlite_mobilenet_v2_coco.config
Checkpoints directory: ../../output/plate_detection/checkpoints/ssdlite_mobilenet_v2_coco_2018_05_09
Frozen model file:     ../../output/plate_detection/fine_tuned_models/ssdlite_mobilenet_v2_coco_2018_05_09\frozen_inference_graph.pb
TFLite model file:     ../../output/plate_detection\glpd-model-float.tflite


# Train Traffic Light Detection Model

**NOTE:** If you have already trained the model, you can skip the training- and export steps and proceed directly with testing...

### Download Pretrained Model

**NOTE:** This step only needs to be done once if the pretrained model has not been downloaded yet!

In [10]:
MODEL_ZIP = os.path.join(PRETRAIND_MODELS_DIR, MODEL_FILE)
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_ZIP)
tar_file = tarfile.open(MODEL_ZIP)
tar_file.extractall(os.path.join(os.getcwd(), PRETRAIND_MODELS_DIR))
tar_file.close()

### Train Finetuned Model

Starts the training. This step may take several hours, depending on the computing power of your computer.
You can monitor the training process using tensorboard tensorboard --logdir=%CHECKPOINTS_DIR% and stop the training if once the desired accuracy has been achieved.

In [8]:
%run model_main.py --pipeline_config_path={PIPELINE_CONFING_FILEPATH} --model_dir={CHECKPOINTS_DIR}


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
INFO:tensorflow:Maybe overwriting load_pretrained: True
INFO:tensorflow:Ignoring config override key: load_pretrained
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to_tpu False
INFO:tensorflow:Using config: {'_model_dir': '../../output/plate_detection/checkpoints/ssdlite_mobilenet_v2_coco_2018_05_09', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '

INFO:tensorflow:Restoring parameters from ../../output/plate_detection/checkpoints/ssdlite_mobilenet_v2_coco_2018_05_09\model.ckpt-929
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
creating index...
index created!
INFO:tensorflow:Loading and preparing annotation results...
INFO:tensorflow:DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.220
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.581
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.068
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.279
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.249
 Average Recal

INFO:tensorflow:global_step/sec: 1.56433
INFO:tensorflow:loss = 1.1049042, step = 3300 (63.963 sec)
INFO:tensorflow:global_step/sec: 1.57292
INFO:tensorflow:loss = 1.3004199, step = 3400 (63.539 sec)
INFO:tensorflow:global_step/sec: 1.55393
INFO:tensorflow:loss = 1.5349876, step = 3500 (64.396 sec)
INFO:tensorflow:global_step/sec: 1.54842
INFO:tensorflow:loss = 1.0030417, step = 3600 (64.539 sec)
INFO:tensorflow:Saving checkpoints for 3607 into ../../output/plate_detection/checkpoints/ssdlite_mobilenet_v2_coco_2018_05_09\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:Done calling model_fn

INFO:tensorflow:loss = 1.1421664, step = 4700 (63.838 sec)
INFO:tensorflow:global_step/sec: 1.55024
INFO:tensorflow:loss = 1.7200829, step = 4800 (64.471 sec)
INFO:tensorflow:global_step/sec: 1.57508
INFO:tensorflow:loss = 1.3275857, step = 4900 (63.538 sec)
INFO:tensorflow:global_step/sec: 1.56178
INFO:tensorflow:loss = 1.25367, step = 5000 (63.980 sec)
INFO:tensorflow:global_step/sec: 1.56199
INFO:tensorflow:loss = 1.2727109, step = 5100 (64.055 sec)
INFO:tensorflow:global_step/sec: 1.56502
INFO:tensorflow:loss = 1.1996188, step = 5200 (63.863 sec)
INFO:tensorflow:global_step/sec: 1.56688
INFO:tensorflow:loss = 0.9918119, step = 5300 (63.863 sec)
INFO:tensorflow:Saving checkpoints for 5381 into ../../output/plate_detection/checkpoints/ssdlite_mobilenet_v2_coco_2018_05_09\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv

INFO:tensorflow:loss = 1.0671122, step = 6500 (63.840 sec)
INFO:tensorflow:global_step/sec: 1.55647
INFO:tensorflow:loss = 1.0933914, step = 6600 (64.205 sec)
INFO:tensorflow:global_step/sec: 1.56294
INFO:tensorflow:loss = 1.0086803, step = 6700 (64.029 sec)
INFO:tensorflow:global_step/sec: 1.58098
INFO:tensorflow:loss = 0.91134095, step = 6800 (63.204 sec)
INFO:tensorflow:global_step/sec: 1.59223
INFO:tensorflow:loss = 1.0346198, step = 6900 (62.848 sec)
INFO:tensorflow:global_step/sec: 1.58065
INFO:tensorflow:loss = 0.82258916, step = 7000 (63.222 sec)
INFO:tensorflow:global_step/sec: 1.58502
INFO:tensorflow:loss = 1.151697, step = 7100 (63.147 sec)
INFO:tensorflow:Saving checkpoints for 7162 into ../../output/plate_detection/checkpoints/ssdlite_mobilenet_v2_coco_2018_05_09\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional c

...

INFO:tensorflow:DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.745
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.955
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.864
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.590
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.748
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.813
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.779
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.779
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.779
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.629
 Aver

INFO:tensorflow:loss = 0.8492899, step = 27500 (122.550 sec)
INFO:tensorflow:Saving checkpoints for 27584 into ../../output/plate_detection/checkpoints/ssdlite_mobilenet_v2_coco_2018_05_09\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-03-18T16:40:18Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../../output/plate_detection/checkpoints/ssdlite_mobilenet_v2_coco_2018_05_09\model.ckpt-27584
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
creating index...
i

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 29371: ../../output/plate_detection/checkpoints/ssdlite_mobilenet_v2_coco_2018_05_09\model.ckpt-29371
INFO:tensorflow:global_step/sec: 1.02495
INFO:tensorflow:loss = 0.528726, step = 29400 (97.539 sec)
INFO:tensorflow:global_step/sec: 1.57597
INFO:tensorflow:loss = 0.7492744, step = 29500 (63.502 sec)
INFO:tensorflow:global_step/sec: 1.55337
INFO:tensorflow:loss = 1.0234061, step = 29600 (64.327 sec)
INFO:tensorflow:global_step/sec: 1.56954
INFO:tensorflow:loss = 0.807192, step = 29700 (63.757 sec)
INFO:tensorflow:global_step/sec: 1.57077
INFO:tensorflow:loss = 1.0505508, step = 29800 (63.619 sec)
INFO:tensorflow:global_step/sec: 1.57438
INFO:tensorflow:loss = 1.1056459, step = 29900 (63.549 sec)
INFO:tensorflow:Saving checkpoints for 30000 into ../../output/plate_detection/checkpoints/ssdlite_mobilenet_v2_coco_2018_05_09\model.ckpt.
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (600 secs).
INFO:te

NotFoundError: Failed to create a directory: ../../output/plate_detection/checkpoints/ssdlite_mobilenet_v2_coco_2018_05_09\export\Servo\temp-b'1584551259'; No such file or directory

AttributeError: 'function' object has no attribute 'called'

### Export Finetuned Model
If the training is finished, the model must be exported so that it can be used.

Set CHECKPOINT_NO = 'XXX' where XXX is the number of the last checkpoint file in CHECKPOINTS_DIR, before you start the export.

In [9]:
CHECKPOINT_NO = 30000

!python export_inference_graph.py --pipeline_config_path={PIPELINE_CONFING_FILEPATH} --trained_checkpoint_prefix={CHECKPOINTS_DIR}/model.ckpt-{CHECKPOINT_NO} --output_directory={FINETUNED_MODEL_DIR}

Parsing Inputs...

Instructions for updating:
Use tf.cast instead.



=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              0
-min_occurrence             0
-step                       -1
-order_by                   name
-account_type_regexes       _trainable_variables
-start_name_regexes         .*
-trim_name_regexes          .*BatchNorm.*
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     params
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
param: Number of parameters (in the Variable).

Profile:
node name | # paramet


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
Instructions for updating:
Use `tf.profiler.profile(graph, run_meta, op_log, cmd, options)`. Build `options` with `tf.profiler.ProfileOptionBuilder`. See README.md for details
Instructions for updating:
Use tf.compat.v1.graph_util.remove_training_nodes
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
2020-03-18 18:08:13.671322: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX AVX2
2020-03-18 18:08:13.821694: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found device 0 with properties: 
name: GeForce RTX 2070 major: 7 minor: 5 memoryClockRate(GHz): 1.83
pciBusID: 0000:01:00.0
totalMemory: 8.00GiB freeMemory: 6.59GiB
2020-03-18 18:08:13.822017: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1512] Adding visible

      BoxPredictor_4/BoxEncodingPredictor/biases (24, 24/24 params)
      BoxPredictor_4/BoxEncodingPredictor/weights (1x1x256x24, 6.14k/6.14k params)
    BoxPredictor_4/BoxEncodingPredictor_depthwise (--/2.30k params)
      BoxPredictor_4/BoxEncodingPredictor_depthwise/BatchNorm (--/0 params)
      BoxPredictor_4/BoxEncodingPredictor_depthwise/depthwise_weights (3x3x256x1, 2.30k/2.30k params)
    BoxPredictor_4/ClassPredictor (--/3.08k params)
      BoxPredictor_4/ClassPredictor/biases (12, 12/12 params)
      BoxPredictor_4/ClassPredictor/weights (1x1x256x12, 3.07k/3.07k params)
    BoxPredictor_4/ClassPredictor_depthwise (--/2.30k params)
      BoxPredictor_4/ClassPredictor_depthwise/BatchNorm (--/0 params)
      BoxPredictor_4/ClassPredictor_depthwise/depthwise_weights (3x3x256x1, 2.30k/2.30k params)
  BoxPredictor_5 (--/6.95k params)
    BoxPredictor_5/BoxEncodingPredictor (--/3.10k params)
      BoxPredictor_5/BoxEncodingPredictor/biases (24, 24/24 params)
      BoxPredictor_5/Bo

# Test Traffic Light Detection

Uses the fine tuned traffic light detection model FINETUNED_MODEL_DIR to detect traffic lights in images.

#### Load the (frozen) finetuned model into memory.

In [10]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

### Loading label map
Loads the label map file PATH_TO_LABELS. This file contains a map that assigns category names to the class-indices:

In [13]:
from object_detection.utils import label_map_util

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

print (label_map)
print (categories)
print (category_index)

item {
  name: "GER"
  id: 1
}

[{'id': 1, 'name': 'GER'}]
{1: {'id': 1, 'name': 'GER'}}


In [14]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [16]:
import glob
import matplotlib.image as mpimg

PATH_TO_TEST_IMAGES_DIR = '../../data/plate_detection/test_images'

TEST_IMAGE_PATHS = []
for filename in glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.jpg')):
    TEST_IMAGE_PATHS.append(filename)
    
print ('{} test images found in {}'.format(len(TEST_IMAGE_PATHS), PATH_TO_TEST_IMAGES_DIR))

plt.imshow(mpimg.imread(TEST_IMAGE_PATHS[0]))

5 test images found in ../../data/plate_detection/test_images


In [17]:
from object_detection.utils import visualization_utils as vis_util

# number of samples to test; -1 <=> all
NUM_SAMPLES = 2

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 12)

if NUM_SAMPLES > 0:
    test_images = np.random.choice(TEST_IMAGE_PATHS, size = NUM_SAMPLES, replace=False)
else:
    test_images = TEST_IMAGE_PATHS

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for image_path in test_images:
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections], feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=4)
            plt.figure(figsize=IMAGE_SIZE)
            plt.title(image_path)
            plt.imshow(image_np)
            plt.show()

C:\Users\Andreas\Anaconda3\envs\tf1.13\lib\site-packages\ipykernel_launcher.py:46: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


### Convert to TF Lite

In [21]:
from tensorflow.compat.v1.lite import TFLiteConverter #, Optimize

input_arrays=["image_tensor"]
output_arrays=["detection_boxes","detection_scores","num_detections","detection_classes"]
input_tensor={"image_tensor":[1,300,300,3]}

#converter = TFLiteConverter.from_frozen_graph(PATH_TO_FROZEN_GRAPH, input_arrays, output_arrays, input_tensor)
converter = TFLiteConverter.from_saved_model(FINETUNED_MODEL_DIR + '/saved_model')
#converter.optimizations=[Optimize.DEFAULT]
tflite_model = converter.convert()

open(TFLITE_MODEL_PATH, "wb").write(tflite_model)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.
INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default'}
INFO:tensorflow:input tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: inputs
INFO:tensorflow: tensor name: image_tensor:0, shape: (-1, -1, -1, 3), type: DT_UINT8
INFO:tensorflow:output tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: detection_boxes
INFO:tensorflow: tensor name: detection_boxes:0, shape: (-1, 10, 4), type: DT_FLOAT
INFO:tensorflow:Tensor's key in saved_model's tensor_map: detection_scor

ValueError: None is only supported in the 1st dimension. Tensor 'image_tensor' has invalid shape '[None, None, None, 3]'.

In [20]:
from tensorflow.compat.v1.lite import Interpreter

# Load TFLite model and allocate tensors.
interpreter = Interpreter(model_path=PATH_TO_FROZEN_GRAPH)
interpreter.allocate_tensors()
# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on random input data
input_shape = input_details[0]['shape']
output_shape = output_details[0]['shape']
print("Input Shape:  {}".format(input_shape))
print("Input Details: {}".format(input_details))
print("Output Details: {}".format(output_details))
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print("Output: {}".format(output_data))

ValueError: Model provided has model identifier 'Cons', should be 'TFL3'
